# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию.

In [1]:
import pandas as pd # импортируйте библиотеку pandas

In [2]:
data  = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')# прочитайте csv-файл

In [3]:
data.head(20)# ваш код здесь

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()# ваш код здесь

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведим количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.

In [5]:
print(data.isna().sum()) # ваш код здесь

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В двух столбцах есть пропущенные значения. Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (d) и есть пропуски в total_income 
    data.loc[(data['income_type'] == d) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == d), 'total_income'].median()# ваш код здесь

### Обработка аномальных значений

Обработаем значения в столбце `days_employed`: замените все отрицательные значения положительными с помощью метода abs()

In [7]:
data['days_employed'] = data['days_employed'].abs()# ваш код здесь

Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.

In [8]:
income_type_day_median = data.groupby('income_type')['days_employed'].median()# ваш код здесь
income_type_day_median

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Выведите перечень уникальных значений столбца `children`

In [9]:
data['children'].unique()# ваш код здесь

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`

In [10]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]# ваш код здесь

### Удаление пропусков

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`

In [11]:
for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (d) и есть пропуски в total_income 
    data.loc[(data['income_type'] == d) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == d), 'days_employed'].median()# ваш код здесь# ваш код здесь

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.

In [12]:
print(data.isna().sum())# ваш код здесь

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [13]:
data['total_income'] = data['total_income'].astype('int')# ваш код здесь

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()# ваш код здесь

Проверим дубликаты.

In [15]:
data.duplicated().sum()# посчитайте дубликаты

71

Удалим дубликаты.

In [16]:
data = data.drop_duplicates() # удалите дубликаты

### Категоризация данных

Категоризируем клиентов по уровню дохода.

In [17]:
def categorize_income(total_income):# создайте функцию categorize_income()
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
         return 'C'
    if total_income <= 1000000:
         return 'B'
    return 'A'    

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)# примените функцию методом apply()

Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [19]:
print(data['purpose'].unique())# ваш код здесь

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Категоризируем клиентов по цели кредита.

In [20]:
def categorize_purpose(purpose):# создайте функцию categorize_purpose()
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)# примените функцию методом apply()

## Исследуем данные и ответим на вопросы.

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Вывод:** Из полученных расчетов видно, что заемщиков, у которых имеется 5 детей очень малое количество и делать выводы о возможности ими возвращать кредиты не стоит. Основываясь на оставшихся данных можно сделать вывод, что вероятность возврата кредита заемщиками,которые не имеют детей самая низкая. Предоставление кредитов данной категории заемщиков наименее рискована. По таблице можно заметить, что с увеличением количества детей, вероятность возврата кредита падает. Из этих данных выбиваются заемщики с 3 детьми. Но возможно из-за короткого времени мониторинга процесса кредитования и малого количества предоставленных кредитов заемщиков с 3 детьми в период проводимого мониторинга их процент невыплат ниже, чем у других категорий заемщиковю Если продолжать мониторинг заемщиков с 3 детьми, скорее всего процент невыплаченных кредитов в данной группе увеличится.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


**Вывод:** Исходя из расчетов, существет зависимость семейного положения и возврата кредита. Заемщики не имеющие никакого официального семейного положения имеют наименьший показатель по выплатам кредита и предоставление кредитов данной категории заемщиков имеет большие риски. В свою очередь заемщики, которые пребывали в браке какое-то время показали себя как наиболее добросовестные заемщики.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
total_income_category,,,
E,22,2,9.09%
C,15921,1353,8.50%
A,25,2,8.00%
B,5014,354,7.06%
D,349,21,6.02%


**Вывод:** Исходя из расчетов, основные заемщики представляют собой группы с доходом B и C. Остальные составляют менее 2%. Если взять данные основных групп за основу для анализ, то явно прослеживается зависимость уровня дохода и возврата средств. Категория B с доходом 200001 - 1000000 имеет 7.06% неоплаченных кредитов, в то время как категория С с доходом 50001 - 200000 имеет 8.50% неоплаченных кредитов.

### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'), ascending = False)\
    .style.format({('mean', 'debt') : '{:.2%}'})

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


**Вывод:** В полученых данных каждая из целей кредита существенна. Информация в таблице полностью отображает влияние цели кредита на его возврат.

### Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* В исходных данных имелись пропуски в столбцах 'days_employed', что соответствует общему трудовому стажу и в столбце 'total_income', что соответствует ежемесячному доходу. Пропуски в общем трудовом стаже могли быть вызваны как отсутствием документов, подтверждающими его так и отсутствием трудового стажа как такового. Поэтому данные могли не вноситься в данный столбец. Отсутствие данных о ежемесячном доходе могли были так же быть вызваны отсутствием документов подтверждающими доход или отсутвием его вообще на момент получния кредита.

### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Ответ: Заполнить пропуски медианным значением — лучшее решение для количественных переменных, так как когда есть оторванные от основной массы лидеры по показателям к какой-либо группе, их результаты сильно смещают значение среднего вверх. Вот почему оценивать предпочтения широкого круга заемщиков лучше медианой.

## Общий вывод

Напишите ваш общий вывод. Исходя из исследования надежности заемщиков можно сделать вывод, что на выплату кредита влияют множество факторов. В худшую сторону на возврат кредита влияет количество детей, а имнно, чем больше у заемщика детей - тем менее он надежен. Семейное положение так же является показателем надежности. Чем дольше человек находился в семейном статусе тем больше вероятность погашения им кредита. Уровень дохода так же сказывается на надежности заемщика, а именно, чем больше доход заемщика, тем более он надежен. Цель получения кредита так же влият на надежность кредита. Лучший показатель в надежности показали кредиты выданные на операции с недвижимостью, за ними кредиты на проведение свадьбы, далее на получение образования и замыкают кредиты на операции с недвижимостью.